In [1]:
!pip3 install tqdm albumentations

In [2]:
import os
import cv2
from tqdm import tqdm
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

import albumentations as A

import torch
from torch import nn
from torch import optim

from torchvision import datasets
from torchvision.transforms import v2
from torchvision import models
from torch.utils.data import DataLoader, Dataset, Subset
import torch.nn.functional as F

from sklearn.model_selection import KFold

from sklearn.metrics import mean_absolute_error, cohen_kappa_score, mean_squared_error
from sklearn.metrics import accuracy_score, f1_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
transform = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True)
])

<h1>Data Augmentation</h1>

In [4]:
# Torchvision library
data_augmentation = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.CenterCrop(size=95),
    v2.RandomVerticalFlip(p=0.5),
    v2.RandomRotation(10),  
    v2.GaussianBlur(kernel_size=5),
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    v2.Normalize(mean=(0.5,), std=(0.5, ))
])

In [ ]:
# Albumentations library
data_augmentation_A = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.CenterCrop(height=90,width=90),
    A.VerticalFlip(p=0.5),
    A.Rotate(10),
    A.GaussianBlur(),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    A.GaussNoise(),
    A.Normalize(mean=(0.5,), std=(0.5, ))
    
])

<h1>Load dataset and Visualize Images</h1>

In [ ]:
# Cell to drop
data_dir = '/kaggle/input/ordinal-classification-uc/ordinal_classification_UC/'
train_data = datasets.ImageFolder(root=os.path.join(data_dir,'train'), transform=transform)
valid_data = datasets.ImageFolder(root=os.path.join(data_dir,'valid'), transform=transform)
test_data = datasets.ImageFolder(root=os.path.join(data_dir,'test'), transform=transform)

In [ ]:
# Cell to drop
ordinal_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
ordinal_dataloader_iter = iter(ordinal_dataloader)
images, labels = next(ordinal_dataloader_iter)

In [5]:
# Load dataset to Cross Validation
data_dir = '/kaggle/input/ordinal-classification-uc-v2/ordinal_classification_UC/'
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

In [6]:
class CustomDataset(Dataset):
    def __init__(self, aOriginalImages, aLabels):
        self.original_images = aOriginalImages
        self.labels = aLabels

    def __len__(self):
        return len(self.original_images)
        
    def __getitem__(self, idx):
        imgs = self.original_images[idx]
            
        labels = self.labels[idx]
        
        return imgs, labels

df_images, df_labels = next(iter(DataLoader(dataset, batch_size=32, shuffle=True)))
dataset_1 = CustomDataset(df_images,df_labels)

In [7]:
NUM_ORDINAL_CLASSES = 4

In [ ]:
def imshow(axe,img):
    npimg = img.numpy()
    axe.imshow(np.transpose(npimg, (1, 2, 0)))
    axe.axis('off')

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 15)) 
class_names = train_data.classes
count = 0
previous_label = []
for img, label in zip(images, labels):
    if count == 0:
        previous_label.append(class_names[label])
        axes[0].set_title(class_names[label])
        imshow(axes[0],img)
        count += 1
    elif class_names[label] not in previous_label:
        previous_label.append(class_names[label])
        axes[count].set_title(class_names[label])
        imshow(axes[count],img)
        count += 1

    if count > 3: break
plt.show()

<h1>Ordinal Classification</h1>

In [8]:
def create_binary_labels_torch(aOrdinalLabels, aClass):
    num_samples = len(aOrdinalLabels)
    binary_labels = []
    labels_k = torch.zeros(num_samples, dtype=torch.float32)
    labels_k[aOrdinalLabels >= aClass] = 1.0
    binary_labels.append(labels_k.unsqueeze(1))
    return torch.cat(binary_labels, dim=1)

In [9]:
class OrdinalDataset(Dataset):
    def __init__(self, aImages, aBinaryLabels, aTransform=None):
        self.images = aImages
        self.binary_labels = aBinaryLabels
        self.transform = aTransform

    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        img = self.images[idx]
        image = img
        if self.transform is not None:
            image = self.transform(img)
            
        labels = self.binary_labels[idx]
        
        return image, labels

In [10]:
class OrdinalClassifier(nn.Module):
    def __init__(self, aNumOrdinalClasses=4):
        super(OrdinalClassifier, self).__init__()

        self.backbone = models.resnet50(weights='DEFAULT')
            
        feature_dim = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        
        self.backbone.drop_layer_1 = nn.Dropout(p=0.20)
        self.backbone.dense_layer_1 = nn.Linear(feature_dim,256)
        self.backbone.drop_layer_2 = nn.Dropout(p=0.20)
        self.backbone.dense_layer_2 = nn.Linear(self.backbone.dense_layer_1.out_features,128)
        self.backbone.ordinal_head = nn.Linear(self.backbone.dense_layer_2.out_features,1)

    def forward(self, x):
        features = self.backbone(x)
        output_drop_layer_1 = self.backbone.drop_layer_1(features)
        output_dense_layer_1 = F.relu(self.backbone.dense_layer_1(output_drop_layer_1))
        output_drop_layer_2 = self.backbone.drop_layer_2(output_dense_layer_1)
        output_dense_layer_2 = F.relu(self.backbone.dense_layer_2(output_drop_layer_2))
        ordinal_output = F.sigmoid(self.backbone.ordinal_head(output_dense_layer_2))
        
        return ordinal_output

In [11]:
class EarlyStopping:
    def __init__(self, aPatience, aDelta=0.01):
        self.patience = aPatience
        self.delta = aDelta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.best_model_state = None

    def __call__(self, val_loss, model):

        if self.best_score is None:
            self.best_score = val_loss
            self.best_model_state = model.state_dict()
        elif val_loss > self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0

    def load_best_model(self, model):
        model.load_state_dict(self.best_model_state)

In [12]:
def get_init(aInstantClass,aTrainSubset,aTestSubset):
    train_images, train_labels = next(iter(DataLoader(aTrainSubset, batch_size=32, shuffle=True)))
    binary_train_labels = create_binary_labels_torch(train_labels,aInstantClass)
    train_dataset = OrdinalDataset(train_images,binary_train_labels,data_augmentation)
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    valid_images, valid_labels = next(iter(DataLoader(aTestSubset, batch_size=32, shuffle=True)))
    binary_valid_labels = create_binary_labels_torch(valid_labels,aInstantClass)
    valid_dataset = OrdinalDataset(valid_images, binary_valid_labels)
    valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

    # Defining model, Loss function, and Optimizer
    model = OrdinalClassifier()
    model.to(device)
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    early_stopping = EarlyStopping(4)

    return train_dataloader,valid_dataloader,model,criterion,optimizer,early_stopping

In [ ]:
# Cell to drop
def get_init_test(aInstantClass):
    test_images, test_labels = next(iter(DataLoader(test_data, batch_size=32, shuffle=True)))
    binary_test_labels = create_binary_labels_torch(test_labels,aInstantClass)
    test_dataset = OrdinalDataset(test_images, binary_test_labels)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    return test_dataloader

In [13]:
K_FOLDS = 10
kFold = KFold(n_splits=K_FOLDS, shuffle=True)

In [14]:
all_acc = []
all_mae = []
all_mse = []
all_kappa = []

In [15]:
def get_predictions(aPredictedBinaryProbs, aLength,aTestData):
    for i_mono in range(NUM_ORDINAL_CLASSES - 2):
        mask = aPredictedBinaryProbs[:, i_mono] < aPredictedBinaryProbs[:, i_mono+1]
        aPredictedBinaryProbs[mask, i_mono+1] = aPredictedBinaryProbs[mask, i_mono]
    
    ordinal_class_probas = torch.zeros((aLength,NUM_ORDINAL_CLASSES), dtype=torch.float32)
    ordinal_class_probas[:, 0] = 1 - aPredictedBinaryProbs[:, 0]
    for k in range(1,NUM_ORDINAL_CLASSES - 1):
        ordinal_class_probas[:, k] = aPredictedBinaryProbs[:, k-1] - aPredictedBinaryProbs[:, k]
        
    if NUM_ORDINAL_CLASSES > 1:
        ordinal_class_probas[:, NUM_ORDINAL_CLASSES - 1] = aPredictedBinaryProbs[:, NUM_ORDINAL_CLASSES - 2]
    
    ordinal_class_probas = np.clip(ordinal_class_probas, 0, 3)
    row_sums = ordinal_class_probas.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1
    ordinal_class_probas = ordinal_class_probas / row_sums
    y_pred_ordinal = np.argmax(ordinal_class_probas, axis=1)
    
    _, y_test = next(iter(DataLoader(aTestData, batch_size=32, shuffle=True)))

    return y_test, y_pred_ordinal

In [ ]:
binary_cnn_classifiers = []
for fold, (train_index, test_index) in enumerate(kFold.split(dataset_1)):
    train_subset = Subset(dataset_1,train_index)
    test_subset = Subset(dataset_1,test_index)

    predicted_binary_probs = torch.zeros((len(test_index),NUM_ORDINAL_CLASSES - 1), dtype=torch.float32)
    
    for k in range(0,NUM_ORDINAL_CLASSES - 1):
        print(f'It will be processed the class {k+1}')
        num_epochs = 150
        train_dataloader,valid_dataloader,model,criterion,optimizer,early_stopping = get_init(k+1,train_subset, test_subset)

        for epoch in range(num_epochs):
            model.train()
            train_loss = 0.0
        
            for images, ordinal_labels in tqdm(train_dataloader, desc='Train: '):
                images, ordinal_labels = images.to(device), ordinal_labels.to(device)
        
        
                optimizer.zero_grad()
                ordinal_pred = model(images)
                loss = criterion(ordinal_pred, ordinal_labels)
                loss.backward()
                optimizer.step()
        
                train_loss += loss.item() * images.size(0)
            epoch_train_loss = train_loss / len(train_subset)
        
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for valid_images, valid_ordinal_labels in tqdm(valid_dataloader, desc='validate: '):
                    valid_images, valid_ordinal_labels = valid_images.to(device), valid_ordinal_labels.to(device)
            
                    valid_ordinal_pred = model(valid_images)
                    validation_loss = criterion(valid_ordinal_pred, valid_ordinal_labels)
                    predicted_binary_probs[:,k] = valid_ordinal_pred.cpu()[:,0]
                    
                    valid_loss += validation_loss.item() * valid_images.size(0)
            epoch_val_loss = valid_loss / len(test_subset)
        
        
            print(f"Epoch {epoch+1}/{num_epochs}: Train_Loss: {epoch_train_loss}\tValid_Loss: {epoch_val_loss}")
        
            early_stopping(epoch_val_loss, model)
            if early_stopping.early_stop: break

    y_true, y_pred = get_predictions(predicted_binary_probs,len(test_index),test_subset)
    all_acc.append(accuracy_score(y_true, y_pred))
    all_mae.append(mean_absolute_error(y_true, y_pred))
    all_mse.append(mean_squared_error(y_true, y_pred))
    all_kappa.append(cohen_kappa_score(y_true, y_pred, weights='quadratic'))

    torch.cuda.empty_cache()
    print('Cleaned GPU memory!!')

It will be processed the class 1


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 168MB/s] 
validate: 100%|██████████| 1/1 [00:00<00:00,  6.39it/s]


Epoch 1/150: Train_Loss: 0.6952852606773376	Valid_Loss: 0.6855770349502563


validate: 100%|██████████| 1/1 [00:00<00:00, 41.16it/s]


Epoch 2/150: Train_Loss: 0.6931763887405396	Valid_Loss: 0.6856117844581604


...

validate: 100%|██████████| 1/1 [00:00<00:00, 67.73it/s]


Epoch 7/150: Train_Loss: 0.6891164183616638	Valid_Loss: 0.6867707967758179


validate: 100%|██████████| 1/1 [00:00<00:00, 62.03it/s]


Epoch 8/150: Train_Loss: 0.6846771240234375	Valid_Loss: 0.6850701570510864


Train: 100%|██████████| 1/1 [00:00<00:00,  6.23it/s]

In [20]:
print(f"Accuracy: {np.mean(all_acc):.4f}")
print(f"Mean Absolute Error: {np.mean(all_mae):.4f}")
print(f"Mean Squared Error: {np.mean(all_mse):.4f}")
print(f"Cohen's Kappa: {np.mean(all_kappa):.4f}")

Accuracy: 0.3917
Mean Absolute Error: 0.8417
Mean Squared Error: 1.4750
Cohen's Kappa: -0.0700


In [ ]:
# Cell to drop
predicted_binary_probs = torch.zeros((32,NUM_ORDINAL_CLASSES - 1), dtype=torch.float32)
for k in range(0,NUM_ORDINAL_CLASSES - 1):
    test_dataloader = get_init_test(k+1)
    binary_classifier = binary_cnn_classifiers[k]
    binary_classifier.eval()
    with torch.no_grad():
        for images, labels in tqdm(test_dataloader, desc='Test: '):
            test_images = images.to(device)
            outputs = binary_classifier(test_images)
            predicted_binary_probs[:,k] = outputs.cpu()[:,0]

In [ ]:
# Cell to drop
mae = mean_absolute_error(y_test, y_pred_ordinal)
mse = mean_squared_error(y_test, y_pred_ordinal)
accuracy = accuracy_score(y_test, y_pred_ordinal)
kappa = cohen_kappa_score(y_test, y_pred_ordinal, weights='quadratic')

print(f"Accuracy: {accuracy:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")

<h1>Ordinal classification + Fine-Grained</h1>

In [ ]:
class OrdinalFineGrainedDataset(Dataset):
    def __init__(self, aImages, aBinaryLabels, aOrdinalClasses, aTransform=None):
        self.images = aImages
        self.binary_labels = aBinaryLabels
        self.ordinal_classes = aOrdinalClasses
        self.transform = aTransform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        image = img
        if self.transform is not None:
            image = self.transform(img)
            
        labels = self.binary_labels[idx]
        fine_class = self.ordinal_classes[idx]

        return image, fine_class, labels

In [ ]:
class OrdinalFineGrainedNet(nn.Module):
    def __init__(self, aNumClasses=4, aNumOrdinaLevels=4):
        super(OrdinalFineGrainedNet, self).__init__()

        self.backbone = models.resnet50(weights='DEFAULT')
        feature_dim = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        self.backbone.fine_class_head = nn.Linear(feature_dim, aNumClasses)
        self.backbone.ordinal_head = nn.Linear(feature_dim, 1)

    def forward(self, x):
        features = self.backbone(x)

        fine_out = F.softmax(self.backbone.fine_class_head(features), dim=1)
        ordinal_out = F.sigmoid(self.backbone.ordinal_head(features))

        return fine_out, ordinal_out

In [ ]:
class OrdinalFineGrainedLoss(nn.Module):
    def __init__(self, aAlpha=1.0, aBeta=1.0):
        super().__init__()
        self.alpha = aAlpha  # Weight for ordinal loss
        self.beta = aBeta  # Weight for fine-grained loss
        self.class_loss = nn.CrossEntropyLoss()
        self.ordinal_loss = nn.BCELoss()  # Binary cross-entropy for ordinal labels

    def forward(self, fine_pred, fine_target, ordinal_pred, ordinal_target):
        loss_fine = self.class_loss(fine_pred, fine_target)  # Fine classification loss
        loss_ordinal = self.ordinal_loss(ordinal_pred, ordinal_target)  # Ordinal loss
        return self.alpha * loss_fine + self.beta * loss_ordinal

In [ ]:
# Defining model, Loss function, and Optimizer
model = OrdinalFineGrainedNet()
model.to(device)
criterion = OrdinalFineGrainedLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# early_stopping = EarlyStopping(patience=5, delta=0.01)

In [ ]:
binary_train_labels = create_binary_labels_torch(labels)
train_dataset = OrdinalFineGrainedDataset(images, binary_train_labels, labels, data_augmentation)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)


valid_images, valid_labels = next(iter(DataLoader(valid_data, batch_size=32, shuffle=True)))
binary_valid_labels = create_binary_labels_torch(valid_labels)
valid_dataset = OrdinalFineGrainedDataset(valid_images, binary_valid_labels, valid_labels)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=True)


test_images, test_labels = next(iter(DataLoader(test_data, batch_size=32, shuffle=True)))
binary_test_labels = create_binary_labels_torch(test_labels)
test_dataset = OrdinalFineGrainedDataset(test_images, binary_test_labels, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for images, fine_labels, ordinal_labels in tqdm(train_dataloader, desc='Train: '):
        images, fine_labels, ordinal_labels = images.to(device), fine_labels.to(device), ordinal_labels.to(device)
        
        optimizer.zero_grad()
        fine_pred, ordinal_pred = model(images)
        loss = criterion(fine_pred, fine_labels, ordinal_pred, ordinal_labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for valid_images, valid_fine_labels, valid_ordinal_labels in tqdm(valid_dataloader, desc='validate: '):
            valid_images, valid_fine_labels, valid_ordinal_labels = valid_images.to(device), valid_fine_labels.to(device), valid_ordinal_labels.to(device)
    
            valid_fine_pred, valid_ordinal_pred = model(valid_images)
            validation_loss = criterion(valid_fine_pred, valid_fine_labels, valid_ordinal_pred, valid_ordinal_labels)
    
            valid_loss = validation_loss.item() * valid_images.size(0)


    print(f"Epoch {epoch+1}/{num_epochs}: Train_Loss: {train_loss / len(train_dataloader)}\tValid_Loss: {valid_loss / len(valid_dataloader)}")

In [ ]:
model.eval()
all_predicted_ordinal = []
all_true_ordinal = []
all_predicted_binary_probs = []
all_predicted_fine_grained = []

with torch.no_grad():
    for test_images, test_fine_labels, test_ordinal_labels in tqdm(test_dataloader, desc='Test: '):
        test_images = test_images.to(device)
        predicted_fine_grained, predicted_ordinal = model(test_images)
        predicted_ordinal_batch = predict_ordinal_torch(predicted_ordinal.cpu(), 0.5)
        all_predicted_ordinal.extend(predicted_ordinal_batch.numpy())
        all_true_ordinal.extend(test_labels)
        all_predicted_binary_probs.extend(predicted_ordinal.cpu().numpy())
        all_predicted_fine_grained.extend(torch.argmax(predicted_fine_grained.cpu(), dim=1))

all_predicted_ordinal = np.array(all_predicted_ordinal)
all_true_ordinal = np.array(all_true_ordinal)
all_predicted_binary_probs = np.array(all_predicted_binary_probs)
all_predicted_fine_grained = np.array(all_predicted_fine_grained)

In [ ]:
accuracy = accuracy_score(all_true_ordinal, all_predicted_fine_grained)
mae = mean_absolute_error(all_true_ordinal, all_predicted_ordinal)
kappa = cohen_kappa_score(all_true_ordinal, all_predicted_ordinal, weights='quadratic')

print(f"Accuracy: {accuracy:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")